In [10]:
def data_crawling(year, num_top_batter=9):
  # Parameter
  # 원하는 연도 입력
  year = year
  # 데이터를 수집할 타자 중 타석 수 상위 명수
  num_top_batter = 9

  # Team List
  # 원하는 연도에 등록된 팀을 미리 넣어두어야함 (추가 가능)
  Team_2016 = ['NC','kt','LG','두산','히어로즈','KIA','롯데','삼성','SK','한화']
  Team_2017 = ['NC','kt','LG','두산','히어로즈','KIA','롯데','삼성','SK','한화']
  Team_2018 = ['NC','kt','LG','두산','히어로즈','KIA','롯데','삼성','SK','한화']
  Team_2019 = ['NC','kt','LG','두산','키움','KIA','롯데','삼성','SK','한화']
  Team_2020 = ['NC','kt','LG','두산','키움','KIA','롯데','삼성','SK','한화']
  Team_2021 = ['kt','삼성','LG','SSG','NC','키움','두산','롯데','KIA','한화']

  # Team Dictionary
  # Team List를 딕셔너리에 넣음 (추가 가능)
  TeamDic = {'2016':Team_2016, '2017':Team_2017, '2018':Team_2018, '2019':Team_2019, '2020':Team_2020, '2021':Team_2021}

  ###############
  # BeautifulSoup

  import numpy as np
  import pandas as pd
  import requests
  from bs4 import BeautifulSoup
  from tqdm import tqdm

  # 데이터를 모을 dataframe 생성
  data = pd.DataFrame(columns=['day', 'vs', 'inn', 'pitcher', 'batter', 'P', 'result', 'before', 'after', 'LEV', 'REa', 'WPs','WPe', 'WPa'])

  batter_data = pd.DataFrame(columns=['Team','name'])

  for Team in tqdm(TeamDic[str(year)]): # 선택한 연도의 팀을 하나씩 가져옴
    
    # 스탯티즈 시즌기록실 페이지에서 연도, 팀 선택하고, 타석순으로 정렬
    wp = requests.get("http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys=" + str(year) + "&ye=" + str(year) + "&se=0&te=" + Team + "&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=TPA&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=")
    soup = BeautifulSoup(wp.text, "html.parser")
    
    # 현재 팀의 타석 수 상위 num_top_batter 명
    tmp_batter_list = np.empty(num_top_batter,dtype=object)
    tmp_team = np.array([Team]*num_top_batter) # 타석 수 상위 타자 팀명 저장
    tmp_batter_list_href = np.empty(num_top_batter,dtype=object)
    tmp_batter_ps=0
    for batter in soup.find_all('tr')[2:11]:
      tmp_batter_list[tmp_batter_ps] = batter.find_all('td')[1].text # 타석 수 상위 타자 이름 저장
      tmp_batter_list_href[tmp_batter_ps] = batter.find_all('a')[0].attrs['href'] # 타석 수 상위 타자 주소 저장
      tmp_batter_ps+=1

    batter_data = pd.concat([batter_data, pd.DataFrame({'Team':tmp_team, 'name':tmp_batter_list})])

    # 타석 수 상위 타자들 중 한명씩 데이터를 모으는 과정
    for top_batter in range(num_top_batter):
      wp=requests.get("http://www.statiz.co.kr/"+tmp_batter_list_href[top_batter]) # 타자 주소로 접근
      soup=BeautifulSoup(wp.text, "html.parser")

      # Playlog 데이터로 접근
      href_by_playlog = soup.select('body > div > div.content-wrapper > div > section.content > div > div:nth-of-type(1) > div > div.col-xs-12.col-sm-8.col-md-8.col-lg-12 > div > div.table-responsive.no-padding.box > table')[0].find_all('a')[6].attrs['href']
      wp=requests.get("http://www.statiz.co.kr/"+href_by_playlog)
      soup=BeautifulSoup(wp.text,"html.parser")

      # year : 원하는 연도로 접근
      href_byDate_year=soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-of-type(1) > div > div:nth-of-type(0) > div')[0].find('a', text=str(year)).attrs['href']
      wp=requests.get("http://www.statiz.co.kr/"+href_byDate_year)
      soup=BeautifulSoup(wp.text,"html.parser")

      ## 데이터가 있는 페이지로 이동 완료
      ## 이제 실질적인 데이터를 모을 차례
      rawdata=soup.select('body > div > div.content-wrapper > div > section.content > div > div:nth-of-type(1) > div > div:nth-of-type(4) > div > div > table')[0]
      # rawdata : 전체 테이블

      # th를 제외한 순수한 경기 데이터 / th는 컬럼명 행이다.
      rawdata_not_th = rawdata.find_all('tr',{'class':['oddrow_stz0','evenrow_stz0']})

      # 행렬만들기
      # rawdata_name = np.array([tmp_batter_list[top_batter]]*len(rawdata_not_th))
      rawdata_day = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_vs = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_inn = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_pitcher = np.zeros(len(rawdata_not_th),dtype=object)
      rawdata_batter = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_P = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_result = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_before = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_after = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_LEV = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_REa = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_WPs = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_WPe = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_WPa = np.zeros(len(rawdata_not_th), dtype=np.float)


      for rawdata_not_th_tr_index in range(len(rawdata_not_th)):
        # 한 행 : rawdata_not_th_tr_index
        rawdata_not_th_td = [text for text in rawdata_not_th[rawdata_not_th_tr_index].stripped_strings]
        # 현재 행의 하나의 값 : rawdata_not_th_td[index]
        # 데이터 입력
        rawdata_day[rawdata_not_th_tr_index] = rawdata_not_th_td[1]
        rawdata_vs[rawdata_not_th_tr_index] = rawdata_not_th_td[2]
        rawdata_result[rawdata_not_th_tr_index] = rawdata_not_th_td[3]
        rawdata_pitcher[rawdata_not_th_tr_index] = rawdata_not_th_td[4]
        rawdata_batter[rawdata_not_th_tr_index] = rawdata_not_th_td[5]
        rawdata_start_memeber[rawdata_not_th_tr_index] = rawdata_not_th_td[6]
        rawdata_result[rawdata_not_th_tr_index] = rawdata_not_th_td[7]
        rawdata_before[rawdata_not_th_tr_index] = rawdata_not_th_td[8]
        rawdata_after[rawdata_not_th_tr_index] = rawdata_not_th_td[9]
        rawdata_LEV[rawdata_not_th_tr_index] = rawdata_not_th_td[10]
        rawdata_REa[rawdata_not_th_tr_index] = rawdata_not_th_td[11]
        rawdata_WPs[rawdata_not_th_tr_index] = rawdata_not_th_td[12]
        rawdata_WPe[rawdata_not_th_tr_index] = rawdata_not_th_td[13]
        rawdata_WPa[rawdata_not_th_tr_index] = rawdata_not_th_td[14]

      tmp_data = pd.DataFrame()
      # tmp_data는 각 선수마다 데이터를 모은 후 data에 머지하기 전에 존재
      tmp_data['day'] = rawdata_day
      tmp_data['vs'] = rawdata_vs
      tmp_data['inn'] = rawdata_inn
      tmp_data['pitcher'] = rawdata_pitcher
      tmp_data['batter'] = rawdata_batter
      tmp_data['P'] = rawdata_start_P
      tmp_data['result'] = rawdata_result
      tmp_data['before'] = rawdata_before
      tmp_data['after'] = rawdata_after
      tmp_data['LEV'] = rawdata_LEV
      tmp_data['REa'] = rawdata_REa
      tmp_data['WPs'] = rawdata_WPs
      tmp_data['WPe'] = rawdata_WPe
      tmp_data['WPa'] = rawdata_WPa

      # 데이터 쌓기
      data = pd.concat([data,tmp_data])

  return data, batter_data


In [11]:
# data_crawling(year, num_top_batter=9)

# data_2018, batter_data_2018 = data_crawling(2018,9)
# data_2019, batter_data_2019 = data_crawling(2019,9)
# data_2020, batter_data_2020 = data_crawling(2020,9)
data_2021, batter_data_2021 = data_crawling(2021,9)

  0%|          | 0/10 [00:04<?, ?it/s]


ValueError: ignored

In [ ]:
data_2018.to_csv("data_2018.csv", encoding='euc-kr', index=False)
data_2019.to_csv("data_2019.csv", encoding='euc-kr', index=False)
data_2020.to_csv("data_2020.csv", encoding='euc-kr', index=False)
data_2021.to_csv("data_2021.csv", encoding='euc-kr', index=False)

batter_data_2018.to_csv("batter_data_2018.csv", encoding='euc-kr', index=False)
batter_data_2019.to_csv("batter_data_2019.csv", encoding='euc-kr', index=False)
batter_data_2020.to_csv("batter_data_2020.csv", encoding='euc-kr', index=False)
batter_data_2021.to_csv("batter_data_2021.csv", encoding='euc-kr', index=False)